In [ ]:
# preprocessing:
def preprocess(fileName):
    rate, data_raw = read_wav(fileName)
    data = (data_raw[:,0]+data_raw[:,1]).astype(np.float32) # stereo of any type -> mono of float32
    data = minmax_scale(data,(-1.,1.))
    fft_image = np.abs(fft.rfft(data,norm='forward'))
    fft_freq = fft.rfftfreq(len(data),1./rate)
    note_volumes = []
    for note in NOTES:        
    
        note_volumes.append(get_note_volume(note,fft_image,fft_freq))
    # print(note_volumes)
    
    return note_volumes

# create a new dataframe with the note volumes
train_note_volumes = pd.DataFrame(columns=NOTES)
test_note_volumes = pd.DataFrame(columns=NOTES)    

for i in tqdm(range(len(train_set))):
    fileName = train_set['File Path'].iloc[i]
    note_volumes = preprocess(fileName)
    # print the shape of note_volumes
    # print(note_volumes.shape)
    for j in range(len(note_volumes)):
        train_note_volumes.at[i, NOTES[j]] = note_volumes[j]

for i in tqdm(range(len(test_set))):
    fileName = test_set['File Path'].iloc[i]
    note_volumes = preprocess(fileName)
    for j in range(len(note_volumes)):
        test_note_volumes.at[i, NOTES[j]] = note_volumes[j]
        # test_set.iloc[i][NOTES[j]] = note_volumes[j]
        # test_set.at[i, NOTES[j]] = note_volumes[j]


In [ ]:

# summarise the quality column in train_set
train_set['Quality'].describe()

# show each quality in train_set and how many there are
train_set.groupby(['Quality']).count()


# how many nan values are there in the train_set?

# add the qualities to the note_volumes dataframe
train_note_volumes['Quality'] = train_set['Quality']
test_note_volumes['Quality'] = test_set['Quality']

len(train_note_volumes)


# print(train_set)


In [ ]:

from sklearn.preprocessing import MinMaxScaler
# Assuming df is your DataFrame and it's already been normalized row-wise

# Create a scaler object
scaler = MinMaxScaler()

# Fit and transform the data but not for the 'File Path', Inversion, Root Note, Quality, and Chord Name columns

columns_to_scale = train_set.drop(columns=['File Path', 'Quality', 'Root Note', 'Octave', 'Inversion']).columns
train_set[columns_to_scale] = scaler.fit_transform(train_set[columns_to_scale])

print(train_set.head())

In [ ]:
print(train_note_volumes['Quality'].unique())
print(len(train_set))
# remove the chords that have nan values in train_set
train_set = train_set.dropna()

In [ ]:
# run a CNN on the data, using the note volumes as the input and the chord quality as the output
# Path: stage_3_cnn_classifier_part_2.ipynb

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

num_classes = 4
height = 12
width = 1
channels = 1

model = Sequential()

# Add Convolutional layers
model.add(Conv1D(32, 3, activation='relu', input_shape=(height, channels)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output to feed into dense layers
model.add(Flatten())

# Add Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Assuming you have already loaded train_note_volumes and labels from your data
# Make sure to convert labels to categorical

# Convert labels to categorical one-hot encoding
labels_categorical = to_categorical(train_note_volumes['Quality'], num_classes=4)

# make input_data a combination of train_note_volumes and test_note_volumes

input_data = np.concatenate((train_note_volumes.drop(columns=['Quality']), test_note_volumes.drop(columns=['Quality'])), axis=0)

# Reshape input data to have a channel dimension
input_data = input_data.reshape(input_data.shape[0], input_data.shape[1], 1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_data, labels_categorical, test_size=0.2, random_state=42)

# Build a simple CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_data.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))  # For multi-class classification with 4 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(input_data, labels_categorical)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
